# Measuring HIC contact matrices similarity with Neural Networks


### Research Questions


Consequently, we set out to answer the following research questions:

1. Can a neural network be trained to predict biological replicates based on Hi-C contact matrices?
2. Does the knowledge from certain cell types generalize to other cell types?
3. In either case, in which segments of the DNA does it generalize better, if any?


## Methods


### Models


The implementation of the three models proposed is shown below.

In [ ]:
import torch.nn as nn
import torchvision
import torch

class SiameseNetwork(nn.Module):
    """
        Based on https://github.com/pytorch/examples/tree/main/siamese_network
        Siamese network for image similarity estimation.
        The network is composed of two identical networks, one for each input.
        The output of each network is concatenated and passed to a linear layer. 
        The output of the linear layer passed through a sigmoid function.
        `"FaceNet" <https://arxiv.org/pdf/1503.03832.pdf>`_ is a variant of the Siamese network.
        This implementation varies from FaceNet as we use the `ResNet-18` model from
        `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_ as our feature extractor.
        In addition, we aren't using `TripletLoss`, `BCELoss` can do the trick.
    """
    def __init__(self, input_size):
        super(SiameseNetwork, self).__init__()

        self.encoder, encoded_features = self.build_encoder(input_size)

        # add linear layers to compare between the features of the two images
        self.fc = nn.Sequential(
            nn.Linear(encoded_features * 2, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 1),
        )

        self.sigmoid = nn.Sigmoid()

        # initialize the weights
        self.encoder.apply(self.init_weights)
        self.fc.apply(self.init_weights)
        
    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)

    def build_encoder(self, input_size, **kwargs):
       raise NotImplementedError()


    def forward_once(self, x):
        output = self.encoder(x)
        output = output.view(output.size()[0], -1)
        return output

    def forward(self, input1, input2):
        # get two images' features
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)

        # concatenate both images' features
        output = torch.cat((output1, output2), 1)

        # pass the concatenation to the linear layers
        output = self.fc(output)

        # pass the out of the linear layers to sigmoid layer
        output = self.sigmoid(output)
        
        return output

class SiameseNetworkResnetEncoder(SiameseNetwork):
    def __init__(self, input_size):
        super(SiameseNetworkResnetEncoder, self).__init__(input_size)


    def build_encoder(self, input_size):
        # get resnet model
        resnet = torchvision.models.resnet18(pretrained=False)

        # over-write the first conv layer to be able to read HIC images
        # as resnet18 reads (3,x,x) where 3 is RGB channels
        # whereas HIC has (1,x,x) where 1 is a gray-scale channel
        resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
        # remove the last layer of resnet18 (linear layer which is before avgpool layer)
        resnet = torch.nn.Sequential(*(list(resnet.children())[:-1]))

        return resnet, resnet(torch.rand(1, 1, *input_size)).shape[1]

class SiameseNetworkLinearEncoder(SiameseNetwork):
    def __init__(self, input_size):
        super(SiameseNetworkLinearEncoder, self).__init__(input_size)

    def build_encoder(self, input_size):
        return nn.Linear(input_size[0] * input_size[1], 256), 256
     
    def forward_once(self, x):
        output = self.encoder(x.view(x.size()[0], -1))
        return output
    
class SiameseNetworkLeNetEncoder(SiameseNetwork):
    def __init__(self, input_size):
        super(SiameseNetworkLeNetEncoder, self).__init__(input_size)

    def build_encoder(self, input_size):
        encoder = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )

        return encoder, torch.prod(torch.tensor(encoder(torch.rand(1, 1, *input_size)).shape))

### Data


Below find the implementation of the dataset class.

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import torch
import torchvision.transforms as T
from tqdm import tqdm
from pathlib import Path
from typing import List, Tuple, Dict

class HICPairsBioReplicatesDataset(Dataset):
    def __init__(
        self,
        root_dir: str,
        bio_replicates_pairs: List[Tuple[str]],
        non_bio_replicates_pairs: List[Tuple[str]],
        chromosomes: List[str],
    ):
        self.root_dir = Path(root_dir)

        print("Building positive image pairs")
        self.positive_pairs = self.__build_image_pairs(
            bio_replicates_pairs, chromosomes
        )

        print("Building negative image pairs")
        self.negative_pairs = self.__build_image_pairs(
            non_bio_replicates_pairs, chromosomes
        )

        self.all_pairs = self.negative_pairs + self.positive_pairs

        self.transform = T.Compose(
            [
                T.ToTensor(),
            ]
        )

    def __build_image_pairs(self, pairs: List[Tuple[str]], chromosomes: List[str]):
        img_pairs = []
        for exp1, exp2 in pairs:
            print(f"Building image pairs for {exp1} and {exp2}")

            dir1, dir2 = Path(self.root_dir / exp1), Path(self.root_dir / exp2)
            assert dir1.exists() and dir2.exists(), f"{dir1} or {dir2} does not exist"

            if chromosomes is None:
                chromosomes = set(p.name for p in dir1.iterdir()).intersection(
                    set(p.name for p in dir2.iterdir())
                )

            for chr in tqdm(chromosomes):

                chr_dir1, chr_dir2 = dir1 / chr, dir2 / chr
                assert (
                    chr_dir1.exists() and chr_dir2.exists()
                ), f"{chr_dir1} or {chr_dir2} does not exist"

                dir1_imgs, dir2_imgs = set(p.name for p in chr_dir1.iterdir()), set(
                    p.name for p in chr_dir2.iterdir()
                )

                for img_name in dir1_imgs.intersection(dir2_imgs):
                    img_pairs.append((chr_dir1 / img_name, chr_dir2 / img_name))

        return img_pairs

    def __get_extra_info(self, path):
        return {
            "experiment": path.parent.parent.name,
            "window": tuple(map(int, path.stem.split("."))),
            "chr": path.parent.name,
            "path": str(path)
        }

    def __getitem__(self, index):
        src_img1, src_img2 = self.all_pairs[index]
        img1, img2 = self.transform(Image.open(src_img1)), self.transform(Image.open(src_img2))
        return {
            "input1": img1,
            "input2": img2,
            "label": torch.tensor([0 if index < len(self.negative_pairs) else 1], dtype=torch.float32),
            "extra1": self.__get_extra_info(src_img1),
            "extra2": self.__get_extra_info(src_img2),
        }

    def __len__(self):
        return len(self.all_pairs)

## Experiments


The following code defines the training procedures and util functions

In [ ]:
import json
import numpy as np
from torch.utils.data import ConcatDataset, random_split
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_path = "../data/hic_dataset/"

experiment_to_cell_type = {
    "GSM1551552_HIC003": "GM12878",
    "GSM1551554_HIC005": "GM12878",
    "GSM1551569_HIC020": "GM12878",
    "GSM1551599_HIC050": "IMR90",
    "GSM1551600_HIC051": "IMR90",
    "GSM1551604_HIC055": "IMR90",
    "GSM1551607_HIC058": "HMEC",
    "GSM1551608_HIC059": "HMEC",
    "GSM1551609_HIC060": "HMEC",
    "GSM1551614_HIC065": "NHEK",
    "GSM1551615_HIC066": "NHEK",
    "GSM1551618_HIC069": "K562",
    "GSM1551619_HIC070": "K562",
    "GSM1551620_HIC071": "K562",
    "GSM1551624_HIC075": "KBM7",
    "GSM1551625_HIC076": "KBM7",
    "GSM1551626_HIC077": "KBM7",
    "GSM1551629_HIC080": "HUVEC",
    "GSM1551630_HIC081": "HUVEC",
}

all_cells_dataset = HICPairsBioReplicatesDataset(
        data_path,
        [
            # GM12878
            ("GSM1551552_HIC003", "GSM1551554_HIC005"), 
            # IMR90 (CCL-186)
            ("GSM1551599_HIC050", "GSM1551600_HIC051"),
            # HMEC (CC-2551)
            ("GSM1551608_HIC059", "GSM1551609_HIC060"),
            # NHEK (192627)
            ("GSM1551614_HIC065", "GSM1551615_HIC066"),
            # K562 (CCL-243)
            ("GSM1551618_HIC069", "GSM1551619_HIC070"),
            # KBM7
            ("GSM1551625_HIC076", "GSM1551626_HIC077"),
            # HUVEC
            ("GSM1551629_HIC080", "GSM1551630_HIC081")
        ],
        [
            # GM12878
            ("GSM1551552_HIC003", "GSM1551569_HIC020"),
            # IMR90 (CCL-186)
            ("GSM1551599_HIC050", "GSM1551604_HIC055"),
            # HMEC (CC-2551)
            ("GSM1551607_HIC058", "GSM1551608_HIC059"),
            # K562 (CCL-243)
            ("GSM1551618_HIC069", "GSM1551620_HIC071"),
            # KBM7
            ("GSM1551624_HIC075", "GSM1551625_HIC076"),
        ],
        None,
    )
gm12878_dataset = HICPairsBioReplicatesDataset(
        data_path,
        [
            ("GSM1551552_HIC003", "GSM1551554_HIC005"), 
        ],
        [
            ("GSM1551552_HIC003", "GSM1551569_HIC020"),
        ],
        None,
    )
imr90_dataset = HICPairsBioReplicatesDataset(
    data_path,
    [
        ("GSM1551599_HIC050", "GSM1551600_HIC051"),
    ],
    [
        ("GSM1551599_HIC050", "GSM1551604_HIC055"),
    ],
    None,
)
hmec_dataset = HICPairsBioReplicatesDataset(
    data_path,
    [
        ("GSM1551608_HIC059", "GSM1551609_HIC060"),
    ],
    [
        ("GSM1551607_HIC058", "GSM1551608_HIC059"),
    ],
    None,
)
k562_dataset = HICPairsBioReplicatesDataset(
    data_path,
    [
        ("GSM1551618_HIC069", "GSM1551619_HIC070"),
    ],
    [
        ("GSM1551618_HIC069", "GSM1551620_HIC071"),
    ],
    None,
)
kbm7_dataset = HICPairsBioReplicatesDataset(
    data_path,
    [
        ("GSM1551625_HIC076", "GSM1551626_HIC077"),
    ],
    [
        ("GSM1551624_HIC075", "GSM1551625_HIC076"),
    ],
    None
)

datasets_by_cell_type = {
    "gm12878": gm12878_dataset,
    "imr90": imr90_dataset,
    "hmec": hmec_dataset,
    "k562": k562_dataset,
    "kbm7": kbm7_dataset
}


def train_once(model, train_loader, criterion, optimizer):
    print("Training model...")
    model.train()

    running_loss = 0.0
    for i,batch in enumerate(train_loader):
        optimizer.zero_grad()
        input1, input2, label = batch["input1"], batch["input2"], batch["label"]
        output = model(input1.to(device), input2.to(device))
        loss = criterion(output, label.to(device))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        print(f"Batch: {i + 1}/{len(train_loader)}, Loss: {running_loss / (i + 1)}", end="\r")

    return running_loss / len(train_loader)

def eval_once(model, test_data, criteria):
    print("Evaluating model...")
    model.eval()

    metrics = {name: 0.0 for name, _ in criteria}
    with torch.no_grad():
        for i,batch in enumerate(test_data):
            input1, input2, label = batch["input1"], batch["input2"], batch["label"]
            output = model(input1.to(device), input2.to(device))

            for name, criterion in criteria:
                metric_value = criterion(output.squeeze(1).cpu(), label.squeeze(1).cpu())
                metrics[name] += metric_value
            print(f"Batch: {i + 1}/{len(test_data)}", end="\r")

    return {name: metric_value / len(test_data) for name, metric_value in metrics.items()}
    
def train_loop(model, train_loader, eval_loader, test_loader, batch_size, num_epochs, criterion, optimizer, eval_metrics, save_dir):
    # Training loop
    history = {
        "train_loss": [],
        "test_loss": [],
        "test_accuracy": [],
    }

    json.dump({"batch_size": batch_size, "num_epochs": num_epochs, "criterion": str(criterion), "optimizer": str(optimizer), "model": str(type(model))}, open(f"{save_dir}/params.json", "w"), indent=4)


    for epoch in range(num_epochs):
        print(f"Epoch: {epoch + 1}")
        
        history["train_loss"].append(train_once(model, train_loader, criterion, optimizer))
        epoch_val_metrics = eval_once(model, eval_loader, eval_metrics)
        for name, value in epoch_val_metrics.items():
            history[f"test_{name}"].append(value)
        
        checkpoint_path = f"{save_dir}/epoch_{epoch + 1}.pt"
        torch.save(model.state_dict(), checkpoint_path)

        print(f"Train loss: {history['train_loss'][-1]:.4f}, Test loss: {epoch_val_metrics['loss']:.4f} Test accuracy: {epoch_val_metrics['accuracy']:.4f}")
    torch.save(model.state_dict(), f"{save_dir}/final.pt")
    json.dump(history, open(f"{save_dir}/history.json", "w"), indent=4)

    best_checkpoint = save_dir / get_best_checkpoint(history)
    model.load_state_dict(torch.load(best_checkpoint))
    test_metrics = eval_once(model, test_loader, eval_metrics)
    json.dump(test_metrics, open(save_dir / "test_metrics.json", "w"), indent=4)

def get_whole_dataset_split(repro_seed=None):
    train_size, val_size = int(0.6 * len(all_cells_dataset)), int(0.2 * len(all_cells_dataset))
    test_size = len(all_cells_dataset) - train_size - val_size

    generator = torch.Generator().manual_seed(repro_seed) if repro_seed is not None else None
    return random_split(all_cells_dataset, [train_size, val_size, test_size], generator=generator)

def get_dataset_split_by_cell_type(train_cell_types, test_cell_types, repro_seed=None):
    train_dataset = ConcatDataset([datasets_by_cell_type[cell_type] for cell_type in train_cell_types])
    test_dataset = ConcatDataset([datasets_by_cell_type[cell_type] for cell_type in test_cell_types])

    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size

    generator = torch.Generator().manual_seed(repro_seed) if repro_seed is not None else None
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size], generator=generator)
    return train_dataset, val_dataset, test_dataset

def get_best_checkpoint(history):
    epoch = np.argmax(history["test_accuracy"])
    return f"epoch_{epoch + 1}.pt"


def dump_analytics_to_df(dataset, models, experiment_to_cell_type):
    loaded_models = []
    for _, model_exp, SiameseNetworkXEncoder in models:
        exp = Path(model_exp)
        best_checkpoint = get_best_checkpoint(json.load(open(exp / "history.json")))
        model = SiameseNetworkXEncoder((40, 40)).to(device)
        model.load_state_dict(torch.load(exp / best_checkpoint))
        model.eval()
        model.to(device)
        loaded_models.append(model)

    columns = ["cell_type", "chr", "low", "high", "label"] + [model_name for model_name, _, _ in models]

    rows = []
    for tuple in tqdm(dataset):
        input1, input2, label, extra = tuple["input1"], tuple["input2"], tuple["label"], tuple["extra1"]
        row = [experiment_to_cell_type[extra["experiment"]], extra["chr"], extra["window"][0], extra["window"][1], int(label.item())]

        input1, input2 = input1.unsqueeze(0).to(device), input2.unsqueeze(0).to(device)
        for model in loaded_models:
            prediction = int(model(input1, input2) > 0.5)
            row.append(prediction)
        
        rows.append(row)

    return pd.DataFrame(rows, columns=columns)

### Experimental setup 1: Train and evaluate with all the cell types

In [ ]:
import torch
from sklearn.metrics import accuracy_score
from datetime import datetime
from pathlib import Path

checkpoint_dir = "../checkpoints"
batch_size = 800
num_epochs = 30
criterion = torch.nn.BCELoss()
eval_metrics = [("loss", lambda x,y: criterion(x,y).item()), ("accuracy", lambda x, y: accuracy_score((x > 0.5).int(), y))]


for exp_no in range(3):
    # Create data loaders
    train_data, val_data, test_data = get_whole_dataset_split(repro_seed=exp_no)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

    models = []
    for name, SiameseNetworkClass in [
        ("resnet", SiameseNetworkResnetEncoder),
        ("lenet", SiameseNetworkLeNetEncoder),
        ("linear", SiameseNetworkLinearEncoder)
    ]:
        # Create model and optimizer instances
        model = SiameseNetworkClass((40, 40)).to(device)
        optimizer = torch.optim.Adam(model.parameters())

        # Create save directory
        save_dir = Path(f"../checkpoints/all_{name}-{exp_no}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")
        save_dir.mkdir(parents=True)

        # Train model
        train_loop(model, train_loader, val_loader, test_loader, batch_size, num_epochs, criterion, optimizer, eval_metrics, save_dir)

        models.append((name, model))
    
      

### Experiment 2: Train with a subset of the cell types and test with another

(Explanation)

In [ ]:
import torch
from sklearn.metrics import accuracy_score
from datetime import datetime
from pathlib import Path

train_test_cell_types = [
    (["gm12878", "imr90", "hmec", "k562"], ["kbm7"]),
    (["gm12878", "imr90", "hmec", "kbm7"], ["k562"]),
    (["gm12878", "imr90", "k562", "kbm7"], ["hmec"]),
    (["gm12878", "hmec", "k562", "kbm7"], ["imr90"]),
    (["imr90", "hmec", "k562", "kbm7"], ["gm12878"]),
]

checkpoint_dir = "../checkpoints"
batch_size = 800
num_epochs = 30
criterion = torch.nn.BCELoss()
eval_metrics = [("loss", lambda x,y: criterion(x,y).item()), ("accuracy", lambda x, y: accuracy_score((x > 0.5).int(), y))]

for exp_no, (train_cell_types, test_cell_types) in enumerate(train_test_cell_types):
    train_dataset, val_dataset, test_dataset = get_dataset_split_by_cell_type(train_cell_types, test_cell_types, repro_seed=exp_no)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    for name, SiameseNetworkClass in [
        ("resnet", SiameseNetworkResnetEncoder),
        ("lenet", SiameseNetworkLeNetEncoder),
        ("linear", SiameseNetworkLinearEncoder)
    ]:
        # Create model and optimizer instances
        model = SiameseNetworkClass((40, 40)).to(device)
        optimizer = torch.optim.Adam(model.parameters())

        # Create save directory
        save_dir = Path(f"../checkpoints/celltype_{name}-{'-'.join(train_cell_types)}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")
        save_dir.mkdir(parents=True)

        # Train model
        train_loop(model, train_loader, val_loader, test_loader, batch_size, num_epochs, criterion, optimizer, eval_metrics, save_dir)


### Analytics

In [12]:
from pathlib import Path

# Dumping analytics
analytics_dir = Path("../analytics")

In [13]:
exp_no = 0

# All cell types

# get data of all cell types, exp0
train_data, val_data, test_data = get_whole_dataset_split(exp_no)
models = [
    ("resnet", "../checkpoints/all_resnet-0_2023-05-06_13-27-41", SiameseNetworkResnetEncoder),
    ("lenet", "../checkpoints/all_lenet-0_2023-05-06_14-54-12", SiameseNetworkLeNetEncoder),
    ("linear", "../checkpoints/all_linear-0_2023-05-06_16-36-28", SiameseNetworkLinearEncoder),
]

train_df = dump_analytics_to_df(train_data, models, experiment_to_cell_type)
train_df.to_csv(analytics_dir / f"all{exp_no}_train.csv", index=False)
val_df = dump_analytics_to_df(val_data, models, experiment_to_cell_type)
val_df.to_csv(analytics_dir / f"all{exp_no}_val.csv", index=False)
test_df = dump_analytics_to_df(test_data, models, experiment_to_cell_type)
test_df.to_csv(analytics_dir / f"all{exp_no}_test.csv", index=False)

c:\Users\alero\Anaconda3\envs\ht\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\alero\Anaconda3\envs\ht\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
100%|██████████| 102997/102997 [28:41<00:00, 59.84it/s]
c:\Users\alero\Anaconda3\envs\ht\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\alero\Anaconda3\envs\ht\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13

In [18]:
# Different cell types

experiments = [
    (["gm12878", "imr90", "hmec", "k562"], ["kbm7"]),
    (["gm12878", "imr90", "hmec", "kbm7"], ["k562"]),
    (["gm12878", "imr90", "k562", "kbm7"], ["hmec"]),
    (["gm12878", "hmec", "k562", "kbm7"], ["imr90"]),
    (["imr90", "hmec", "k562", "kbm7"], ["gm12878"]),
]

def find_with_prefix(path, prefix):
    return next(p for p in path.iterdir() if p.name.startswith(prefix))

exp_no = 4
train_cell_types, test_cell_types = experiments[exp_no]
exp = "-".join(train_cell_types)

resnet_exp = find_with_prefix(Path("../checkpoints"), f"celltype_resnet-{exp}")
lenet_exp = find_with_prefix(Path("../checkpoints"), f"celltype_lenet-{exp}")
linear_exp = find_with_prefix(Path("../checkpoints"), f"celltype_linear-{exp}")

train_data, val_data, test_data = get_dataset_split_by_cell_type(train_cell_types, test_cell_types, exp_no)
models = [
    ("resnet", f"../checkpoints/{resnet_exp}", SiameseNetworkResnetEncoder),
    ("lenet", f"../checkpoints/{lenet_exp}", SiameseNetworkLeNetEncoder),
    ("linear", f"../checkpoints/{linear_exp}", SiameseNetworkLinearEncoder),
]

train_df = dump_analytics_to_df(train_data, models, experiment_to_cell_type)
train_df.to_csv(analytics_dir / f"celltype_{exp}_train.csv", index=False)
val_df = dump_analytics_to_df(val_data, models, experiment_to_cell_type)
val_df.to_csv(analytics_dir / f"celltype_{exp}_val.csv", index=False)
test_df = dump_analytics_to_df(test_data, models, experiment_to_cell_type)
test_df.to_csv(analytics_dir / f"celltype_{exp}_test.csv", index=False)

c:\Users\alero\Anaconda3\envs\ht\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\alero\Anaconda3\envs\ht\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
100%|██████████| 91500/91500 [25:55<00:00, 58.82it/s]
c:\Users\alero\Anaconda3\envs\ht\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\alero\Anaconda3\envs\ht\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 a

In [92]:
import matplotlib.pyplot as plt
import numpy as np

def get_accuracy_histogram(ax, df, chr, cell_type=None, model="resnet", bins=100):
    df = df[df["chr"] == chr]
    if cell_type is not None:
        df = df[df["cell_type"] == cell_type]

    counts, bins = np.histogram(df["start_pos"], 100)
    corrects, _ = np.histogram(df[df["label"] == df[model]]["start_pos"], bins)
    acc = corrects / counts
    acc = np.nan_to_num(acc, nan=0)
    ax.bar(bins[:-1], acc, width=bins[1] - bins[0])

def get_counts_histogram(ax, df, chr, cell_type=None, label=None, bins=100):
    df = df[df["chr"] == chr]
    if cell_type is not None:
        df = df[df["cell_type"] == cell_type]
    if label is not None:
        df = df[df["label"] == label]

    counts, bins = np.histogram(df["start_pos"], 100)
    ax.bar(bins[:-1], counts, width=bins[1] - bins[0])



In [99]:
%matplotlib qt

from pandas import read_csv

df = read_csv("../analytics/celltype_gm12878-imr90-hmec-k562_test.csv")
df = df.assign(start_pos=lambda x: x.low // 200000)

fig, axes = plt.subplots(4,6)
fig.subplots_adjust(left=0.05, right=0.95, bottom=0.05, top=0.95, wspace=0.3, hspace=0.4)
chrs = [
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
    "10",
    "11",
    "12",
    "13",
    "14",
    "15",
    "16",
    "17",
    "18",
    "19",
    "20",
    "21",
    "22",
    "X",
    "Y",
]
chr_length = {
    "1": 248956422,
    "2": 242193529,
    "3": 198295559,
    "4": 190214555,
    "5": 181538259,
    "6": 170805979,
    "7": 159345973,
    "8": 145138636,
    "9": 138394717,
    "10": 133797422,
    "11": 135086622,
    "12": 133275309,
    "13": 114364328,
    "14": 107043718,
    "15": 101991189,
    "16": 90338345,
    "17": 83257441,
    "18": 80373285,
    "19": 58617616,
    "20": 64444167,
    "21": 46709983,
    "22": 50818468,
    "X": 156040895,
    "Y": 57227415,
}
chr_length = {k: v // 200000 for k, v in chr_length.items()}

axes = axes.flatten()
for chr, ax in zip(chrs, axes):
    get_accuracy_histogram(ax, df, chr, cell_type=None, model="linear")
    # get_counts_histogram(ax, df, chr, cell_type=None)
    ax.set_xlim(0, chr_length[chr])
    ax.set_ylim(0, 1)
    ax.set_title(chr)
plt.show()

C:\Users\alero\AppData\Local\Temp\ipykernel_2172\4227160043.py:13: RuntimeWarning: invalid value encountered in divide
  acc = corrects / counts


In [51]:
from sklearn.metrics import accuracy_score

def get_accuracy(df, chr=None, model="resnet"):
    if chr is not None:
        df = df[df["chr"] == chr]
    return accuracy_score(df["label"], df[model])

In [54]:
for exp in Path("../analytics/").iterdir():
    print(exp.name)
    df = read_csv(exp)
    for chr in chrs + [None]:
        accs = []
        for model in ["resnet", "lenet", "linear"]:
            accs.append(get_accuracy(df, chr, model))
        print(f"{chr} & {round(accs[0], 4)} & {round(accs[1], 4)} & {round(accs[2], 4)} \\\\")

all0_test.csv
1 & 0.9539 & 0.9476 & 0.9296 \\
2 & 0.9628 & 0.9531 & 0.934 \\
3 & 0.9715 & 0.9659 & 0.9493 \\
4 & 0.9707 & 0.955 & 0.941 \\
5 & 0.9614 & 0.9544 & 0.937 \\
6 & 0.9722 & 0.9571 & 0.939 \\
7 & 0.9552 & 0.9505 & 0.9236 \\
8 & 0.9624 & 0.9566 & 0.9265 \\
9 & 0.943 & 0.9282 & 0.9092 \\
10 & 0.9535 & 0.9467 & 0.9207 \\
11 & 0.965 & 0.9475 & 0.9313 \\
12 & 0.9672 & 0.9584 & 0.9508 \\
13 & 0.9736 & 0.9663 & 0.9363 \\
14 & 0.9603 & 0.9283 & 0.9302 \\
15 & 0.9472 & 0.9259 & 0.9188 \\
16 & 0.9315 & 0.924 & 0.9176 \\
17 & 0.9542 & 0.9467 & 0.9478 \\
18 & 0.9632 & 0.9487 & 0.9331 \\
19 & 0.9404 & 0.9348 & 0.9251 \\
20 & 0.9712 & 0.9621 & 0.9516 \\
21 & 0.9356 & 0.9332 & 0.9332 \\
22 & 0.9291 & 0.922 & 0.9267 \\
X & 0.9566 & 0.9414 & 0.9311 \\
Y & 0.7875 & 0.7375 & 0.8125 \\
None & 0.9592 & 0.9485 & 0.9327 \\
all0_train.csv
1 & 0.9873 & 0.9501 & 0.9559 \\
2 & 0.9919 & 0.9533 & 0.9537 \\
3 & 0.9943 & 0.9646 & 0.96 \\
4 & 0.9945 & 0.9628 & 0.9568 \\
5 & 0.9902 & 0.9586 & 0.9547 \\
6 & 0.

In [100]:
for exp in Path("../analytics/").iterdir():
    print(exp.name)
    df = read_csv(exp)
    accs = []
    for model in ["resnet", "lenet", "linear"]:
        accs.append(get_accuracy(df, None, model))
    print(f"Overall & {round(accs[0], 4)} & {round(accs[1], 4)} & {round(accs[2], 4)} \\\\")

all0_test.csv
Overall & 0.9592 & 0.9485 & 0.9327 \\
all0_train.csv
Overall & 0.9899 & 0.9523 & 0.9537 \\
all0_val.csv
Overall & 0.9598 & 0.9471 & 0.933 \\
celltype_gm12878-hmec-k562-kbm7_test.csv
Overall & 0.5549 & 0.5182 & 0.5283 \\
celltype_gm12878-hmec-k562-kbm7_train.csv
Overall & 0.9896 & 0.9922 & 0.977 \\
celltype_gm12878-hmec-k562-kbm7_val.csv
Overall & 0.9808 & 0.9819 & 0.9677 \\
celltype_gm12878-imr90-hmec-k562_test.csv
Overall & 0.6036 & 0.5067 & 0.5525 \\
celltype_gm12878-imr90-hmec-k562_train.csv
Overall & 0.9753 & 0.9521 & 0.9489 \\
celltype_gm12878-imr90-hmec-k562_val.csv
Overall & 0.9355 & 0.9391 & 0.9115 \\
celltype_gm12878-imr90-hmec-kbm7_test.csv
Overall & 0.7389 & 0.5102 & 0.5281 \\
celltype_gm12878-imr90-hmec-kbm7_train.csv
Overall & 0.9531 & 0.9514 & 0.9445 \\
celltype_gm12878-imr90-hmec-kbm7_val.csv
Overall & 0.9352 & 0.9426 & 0.9203 \\
celltype_gm12878-imr90-k562-kbm7_test.csv
Overall & 0.4604 & 0.7025 & 0.6736 \\
celltype_gm12878-imr90-k562-kbm7_train.csv
Overal

In [79]:
all_train_df = read_csv("../analytics/all0_train.csv")
all_val_df = read_csv("../analytics/all0_val.csv")
all_test_df = read_csv("../analytics/all0_test.csv")

for cell_type in ["GM12878", "IMR90", "HMEC", "K562", "KBM7"]:
    total = 0
    for df in [all_train_df, all_val_df, all_test_df]:
        total += len(df[df["cell_type"] == cell_type])
    print(f"{cell_type} & {total}\\\\")

GM12878 & 28618\\
IMR90 & 28610\\
HMEC & 28520\\
K562 & 28495\\
KBM7 & 28751\\


In [82]:
im1 = Path("C:/Users/alero/Documents/School/Spring23/Bio/Project/code/data/hic_dataset/GSM1551552_HIC003/1/0.199999.jpg")
im2 = Path("C:/Users/alero/Documents/School/Spring23/Bio/Project/code/data/hic_dataset/GSM1551552_HIC003/1/200000.399999.jpg")
im3 = Path("C:/Users/alero/Documents/School/Spring23/Bio/Project/code/data/hic_dataset/GSM1551552_HIC003/1/400000.599999.jpg")
im4 = Path("C:/Users/alero/Documents/School/Spring23/Bio/Project/code/data/hic_dataset/GSM1551552_HIC003/1/600000.799999.jpg")
im5 = Path("C:/Users/alero/Documents/School/Spring23/Bio/Project/code/data/hic_dataset/GSM1551552_HIC003/1/800000.999999.jpg")
im6 = Path("C:/Users/alero/Documents/School/Spring23/Bio/Project/code/data/hic_dataset/GSM1551552_HIC003/1/1000000.1199999.jpg")

In [85]:
from PIL import Image
import matplotlib.pyplot as plt

# Load the six images
image1 = Image.open(im1).convert('L')
image2 = Image.open(im2).convert('L')
image3 = Image.open(im3).convert('L')
image4 = Image.open(im4).convert('L')
image5 = Image.open(im5).convert('L')
image6 = Image.open(im6).convert('L')

# Create a new figure and set the size and margins
fig = plt.figure(figsize=(5, 7))
fig.subplots_adjust(left=0.05, right=0.95, bottom=0.05, top=0.95, wspace=0.1, hspace=0.2)

# Add the six images to the figure as subplots
ax1 = fig.add_subplot(3, 2, 1)
ax1.imshow(image1, cmap='gray')
ax2 = fig.add_subplot(3, 2, 2)
ax2.imshow(image2, cmap='gray')
ax3 = fig.add_subplot(3, 2, 3)
ax3.imshow(image3, cmap='gray')
ax4 = fig.add_subplot(3, 2, 4)
ax4.imshow(image4, cmap='gray')
ax5 = fig.add_subplot(3, 2, 5)
ax5.imshow(image5, cmap='gray')
ax6 = fig.add_subplot(3, 2, 6)
ax6.imshow(image6, cmap='gray')

# Remove the axis labels
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
ax4.axis('off')
ax5.axis('off')
ax6.axis('off')

# Show the figure
plt.show()